# Create hashes for pooch

Handling data that haven't been downloaded before

In [12]:
import sys
import importlib
import yaml
sys.path.append('/Users/eddifying/Cloudfree/gitlab-cloudfree/seagliderOG1')
sys.path.append('/Users/eddifying/Cloudfree/gitlab-cloudfree/glidertest')

# python -c "import pooch; pooch.make_registry('/Users/eddifying/Library/Caches/seagliderOG1','seagliderOG1/registry.txt')"

In [13]:
import matplotlib.pyplot as plt
import yaml
from seagliderOG1 import plotters
from seagliderOG1 import readers
from seagliderOG1 import convertOG1
from seagliderOG1 import tools
from seagliderOG1 import fetchers
import xarray as xr
import os
import warnings
import pooch
from seagliderOG1 import vocabularies
warnings.simplefilter("ignore", category=Warning)


In [ ]:
# Names of the data files
# Found these by searching on https://catalog.data.gov/dataset/?q=eriksen&sort=score+desc%2C+name+asc&ext_location=&ext_bbox=&ext_prev_extent=&page=4
servers = [
    # Labrador Sea
    "https://www.ncei.noaa.gov/data/oceans/glider/seaglider/uw/015/20040924/",
    "https://www.ncei.noaa.gov/data/oceans/glider/seaglider/uw/014/20040924/",
    "https://www.ncei.noaa.gov/data/oceans/glider/seaglider/uw/008/20031002/",
    "https://www.ncei.noaa.gov/data/oceans/glider/seaglider/uw/004/20031002/",
    "https://www.ncei.noaa.gov/data/oceans/glider/seaglider/uw/016/20050406/",
    # RAPID/MOCHA
    "https://www.ncei.noaa.gov/data/oceans/glider/seaglider/uw/033/20100729/",
    "https://www.ncei.noaa.gov/data/oceans/glider/seaglider/uw/034/20110128/",

    # Either Iceland, Faroes or RAPID/MOCHA
    "https://www.ncei.noaa.gov/data/oceans/glider/seaglider/uw/005/20090829/",
    "https://www.ncei.noaa.gov/data/oceans/glider/seaglider/uw/005/20080606/",
    "https://www.ncei.noaa.gov/data/oceans/glider/seaglider/uw/005/20081106/",
    "https://www.ncei.noaa.gov/data/oceans/glider/seaglider/uw/012/20070831/",
    "https://www.ncei.noaa.gov/data/oceans/glider/seaglider/uw/014/20080214/",
    "https://www.ncei.noaa.gov/data/oceans/glider/seaglider/uw/014/20080222/",
    "https://www.ncei.noaa.gov/data/oceans/glider/seaglider/uw/016/20061112/",
    "https://www.ncei.noaa.gov/data/oceans/glider/seaglider/uw/016/20090605/",
    "https://www.ncei.noaa.gov/data/oceans/glider/seaglider/uw/016/20071113/",
    "https://www.ncei.noaa.gov/data/oceans/glider/seaglider/uw/016/20080607/",
    "https://www.ncei.noaa.gov/data/oceans/glider/seaglider/uw/033/20100518/",
    "https://www.ncei.noaa.gov/data/oceans/glider/seaglider/uw/033/20100903/",
    "https://www.ncei.noaa.gov/data/oceans/glider/seaglider/uw/101/20081108/",
    "https://www.ncei.noaa.gov/data/oceans/glider/seaglider/uw/101/20061112/",
    "https://www.ncei.noaa.gov/data/oceans/glider/seaglider/uw/101/20070609/",
    "https://www.ncei.noaa.gov/data/oceans/glider/seaglider/uw/102/20061112/",
    "https://www.ncei.noaa.gov/data/oceans/glider/seaglider/uw/102/20071113/",
    "https://www.ncei.noaa.gov/data/oceans/glider/seaglider/uw/103/20090223/",
    "https://www.ncei.noaa.gov/data/oceans/glider/seaglider/uw/103/20071113/",
    "https://www.ncei.noaa.gov/data/oceans/glider/seaglider/uw/103/20070218/",
    "https://www.ncei.noaa.gov/data/oceans/glider/seaglider/uw/104/20080214/",
    "https://www.ncei.noaa.gov/data/oceans/glider/seaglider/uw/104/20070901/"
]

for server in servers:
    print(f"Processing server: {server}")

    filenames = fetchers.list_files_in_https_server(server)


    # Base url for the data files
    base_url = server

    # Create a new directory where all the files will be downloaded
    basedir = '/Users/eddifying/Library/Caches/seagliderOG1'
    directory = os.path.join(basedir)
    if not os.path.exists(directory):
        os.makedirs(directory)

    # Download each data file to data_files
    for fname in filenames:
        path = pooch.retrieve(
            url = base_url + fname, known_hash=None, fname=fname, path=directory
        )

    # Create a registry file from the downloaded files
    pooch.make_registry(basedir, "registry_new.txt")

    def update_registry(existing_registry_path, new_registry_path):
        # Read the existing registry file
        with open(existing_registry_path, 'r') as f:
            existing_lines = set(f.readlines())

        # Read the new registry file
        with open(new_registry_path, 'r') as f:
            new_lines = f.readlines()

        # Find lines that are in the new registry but not in the existing registry
        unique_lines = [line for line in new_lines if line not in existing_lines]

        # Append the unique lines to the existing registry file
        with open(existing_registry_path, 'a') as f:
            f.writelines(unique_lines)

        # Remove duplicate lines with None values
        with open(existing_registry_path, 'r') as f:
            lines = f.readlines()

        # Create a dictionary to store the best value for each key
        registry_dict = {}
        for line in lines:
            key, value = line.split()
            if key not in registry_dict or registry_dict[key] == 'None':
                registry_dict[key] = value

        # Write the cleaned registry back to the file
        with open(existing_registry_path, 'w') as f:
            for key, value in registry_dict.items():
                f.write(f"{key} {value}\n")

    # Example usage
    update_registry('../seagliderOG1/seaglider_registry.txt', 'registry_new.txt')

Processing server: https://www.ncei.noaa.gov/data/oceans/glider/seaglider/uw/015/20040924/
Processing server: https://www.ncei.noaa.gov/data/oceans/glider/seaglider/uw/014/20040924/
Processing server: https://www.ncei.noaa.gov/data/oceans/glider/seaglider/uw/008/20031002/
Processing server: https://www.ncei.noaa.gov/data/oceans/glider/seaglider/uw/004/20031002/
Processing server: https://www.ncei.noaa.gov/data/oceans/glider/seaglider/uw/016/20050406/
Processing server: https://www.ncei.noaa.gov/data/oceans/glider/seaglider/uw/033/20100729/
Processing server: https://www.ncei.noaa.gov/data/oceans/glider/seaglider/uw/034/20110128/


SHA256 hash of downloaded file: b6ec66e45866a46aa34b448f09bb84d4becf2be7e862fad738e468995d53c4c9
Use this value as the 'known_hash' argument of 'pooch.retrieve' to ensure that the file hasn't changed if it is downloaded again in the future.
SHA256 hash of downloaded file: 72381f5e1f6cc40da3e2ffda5eb08ef25e664d680babaa7f423d321ae5d6fd61
Use this value as the 'known_hash' argument of 'pooch.retrieve' to ensure that the file hasn't changed if it is downloaded again in the future.
SHA256 hash of downloaded file: 415c778c87bea8e11f1cff286720ef59cf304ef75c291214b34daa704da8cdc7
Use this value as the 'known_hash' argument of 'pooch.retrieve' to ensure that the file hasn't changed if it is downloaded again in the future.
SHA256 hash of downloaded file: 5a0018f3bb260027e236dd5fbe8cfe10ae3e9ff5febb76ba384ea9f8b82ccba3
Use this value as the 'known_hash' argument of 'pooch.retrieve' to ensure that the file hasn't changed if it is downloaded again in the future.
SHA256 hash of downloaded file: bf6d

Processing server: https://www.ncei.noaa.gov/data/oceans/glider/seaglider/uw/005/20090829/


SHA256 hash of downloaded file: 279ea4b74aa1006644ead38a3b8250b08a99dae616028f9b0878f92def152908
Use this value as the 'known_hash' argument of 'pooch.retrieve' to ensure that the file hasn't changed if it is downloaded again in the future.
SHA256 hash of downloaded file: 5f6858401ba62827308f1d33b6e922a8ce1df20afbacdf9fb024ec49a291e6de
Use this value as the 'known_hash' argument of 'pooch.retrieve' to ensure that the file hasn't changed if it is downloaded again in the future.
SHA256 hash of downloaded file: a5265a60ec5606a8ed071df0d70f2278b7c3532399f217bff56186ebb0ba4b1e
Use this value as the 'known_hash' argument of 'pooch.retrieve' to ensure that the file hasn't changed if it is downloaded again in the future.
SHA256 hash of downloaded file: 3d86c39d947f1e7edfa642515668086c1f4bb187f423c9b0f58455fde2895ea1
Use this value as the 'known_hash' argument of 'pooch.retrieve' to ensure that the file hasn't changed if it is downloaded again in the future.
SHA256 hash of downloaded file: 5753